- JPype 1.4.1 --> python 3.11
- http://github.com/jpype-project/jpype/releases

# mecab 테스트

In [1]:
import MeCab

m = MeCab.Tagger()
m.parse('안녕하세요. 테스트 입니다.')

'안녕\tNNG,행위,T,안녕,*,*,*,*\n하\tXSV,*,F,하,*,*,*,*\n세요\tEP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*\n.\tSF,*,*,*,*,*,*,*\n테스트\tNNG,행위,F,테스트,*,*,*,*\n입니다\tVCP+EF,*,F,입니다,Inflect,VCP,EF,이/VCP/*+ᄇ니다/EF/*\n.\tSF,*,*,*,*,*,*,*\nEOS\n'

- konlpy Mecab 테스트

In [2]:
from konlpy.tag import Mecab

tokenizer = Mecab(dicpath='c:/mecab/mecab-ko-dic/').morphs
print(tokenizer('안녕하세요. 테스트 입니다.'))

['안녕', '하', '세요', '.', '테스트', '입니다', '.']


# 데이터 로딩 및 확인

In [3]:
import pandas as pd

train_df = pd.read_csv('./dataset/ratings_train.txt', sep='\t', encoding='utf-8')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
print(train_df.shape)

(150000, 3)


In [5]:
test_df = pd.read_csv('./dataset/ratings_test.txt', sep='\t', encoding='utf-8')
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [6]:
print(test_df.shape)

(50000, 3)


- 긍정(1), 부정(0) 비율 확인

In [7]:
train_df['label'].value_counts()

label
0    75173
1    74827
Name: count, dtype: int64

In [8]:
test_df['label'].value_counts()

label
1    25173
0    24827
Name: count, dtype: int64

- 결측치 확인

In [9]:
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [10]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

# 데이터 전처리

## 결측치 삭제

In [11]:
train_df.dropna(inplace=True)

In [12]:
test_df.dropna(inplace=True)

## id 컬럼 삭제

In [13]:
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

## konlpy, Mecab을 이용한 단어 토큰화

In [14]:
from konlpy.tag import Mecab

mecab = Mecab(dicpath='c:/mecab/mecab-ko-dic/')
tokens = [ mecab.morphs(sentence) for sentence in train_df['document']] # 단어 단위로 토큰화
tokens = list(map(lambda x : ' '.join(x), tokens))  # 단어를 공백을 기준으로 결합하여 문장으로 만들기
tokens[:10]

['아 더 빙 . . 진짜 짜증 나 네요 목소리',
 '흠 . .. 포스터 보고 초딩 영화 줄 . ... 오버 연기 조차 가볍 지 않 구나',
 '너무 재 밓었다그래서보는것을추천한다',
 '교도소 이야기 구먼 . . 솔직히 재미 는 없 다 . . 평점 조정',
 '사이몬페그 의 익살 스런 연기 가 돋보였 던 영화 ! 스파이더맨 에서 늙 어 보이 기 만 했 던 커스틴 던스트 가 너무나 도 이뻐 보였 다',
 '막 걸음마 뗀 3 세 부터 초등 학교 1 학년 생 인 8 살 용 영화 . ㅋㅋㅋ . .. 별반 개 도 아까움 .',
 '원작 의 긴장감 을 제대로 살려 내 지 못했 다 .',
 '별 반개 도 아깝 다 욕 나온다 이응경 길용우 연기 생활 이 몇 년 인지 . . 정말 발 로 해도 그것 보단 낫 겟 다 납치 . 감금 만 반복 반복 . . 이 드라마 는 가족 도 없 다 연기 못 하 는 사람 만 모엿 네',
 '액션 이 없 는데 도 재미 있 는 몇 안 되 는 영화',
 '왜 케 평점 이 낮 은 건데 ? 꽤 볼 만한데 . . 헐리우드 식 화려 함 에 만 너무 길들여져 있 나 ?']

# KNU 한글 감성어 사전 로딩

In [15]:
import json

with open('./dataset/SentiWord_info.json', encoding='utf-8', mode='r') as f:
    sentiword_info = json.load(f)

sentiword_dic = pd.DataFrame(sentiword_info)
print(sentiword_dic.shape)
sentiword_dic.head()

(14843, 3)


,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1


In [16]:
sentiword_dic['polarity'] = sentiword_dic['polarity'].astype(int)

In [17]:
print('감성 최대값:', sentiword_dic['polarity'].max())
print('감성 최소값:', sentiword_dic['polarity'].min())
print('감성 최소값:', sentiword_dic['polarity'].mean())

감성 최대값: 2
감성 최소값: -2
감성 최소값: -0.48258438321094116


In [18]:
sentiword_dic['polarity'].value_counts()

polarity
-1    5029
-2    4797
 2    2597
 1    2266
 0     154
Name: count, dtype: int64

# 토큰화된 문장과 감성사전 비교

In [ ]:
# 이거 실행되는데 되게 오래걸림
sentiment_score = pd.DataFrame(columns=['review','sentiment_score'])

idx = 0
for token in tokens:  # tokens: 형태소 분석을 한 토큰화된 단어 리스트
    senti_score = 0
    for i in range(len(sentiword_dic)):  # sentiword_dic: knu감성사전
        if sentiword_dic['word'][i] in token:  # 토큰화된 단어가 감성사전의 단어에 포함되어 있으면
            senti_score += sentiword_dic['polarity'][i]  # 감성사전의 극성값 누적합
    sentiment_score.loc[idx] = [token, senti_score]  # 위의 데이터 프레임에 값 추가 
    idx += 1

In [ ]:
# 리뷰 감성 값 DataFrame 저장
import joblib

joblib.dump(sentiment_score, './sentiment_score.pkl')

In [ ]:
# 위에꺼 또 안돌려도 됨. 저장되어있는 피클파일 불러오기
sentiment_score = joblib.load('./sentiment_score.pkl')

# 감성 값 긍/부정 레이블 값으로 치환

In [ ]:
sentiment_score['label'] = sentiment_score['sentiment_score'].map(lambda x: 1 if x >=0 else 0)

# 정답과 비교하여 정확도 측정

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

matrix = confusion_matrix(train_df['label', sentiment_score['label'])
print(matrix)

print(f'정확도: {accuracy_score(train_df['label'], sentiment_score['label']):.2f}')

# 추가 분석 사항
- 전처리
    - 한글을 제외한 다른 문자 제거 -> 제거 결과 아무런 값도 남지 않은 레코드 확인 -> 해당 레코드 삭제
    - 불용어 제거: 